In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using ControlSystems
using ForwardDiff
using PyPlot

In [ ]:
#Model parameters
g = 9.81 #m/s^2
m = 1.0 #kg 
ℓ = 0.3 #meters
J = 0.2*m*ℓ*ℓ

#Thrust limits
umin = [0.2*m*g; 0.2*m*g]
umax = [0.6*m*g; 0.6*m*g]

h = 0.05; #time step (20 Hz)

In [ ]:
#Planar Quadrotor Dynamics
function quad_dynamics(x,u)
    θ = x[3]
    
    ẍ = (1/m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/m)*(u[1] + u[2])*cos(θ) - g
    θ̈ = (1/J)*(ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

#Linearized dynamics for hovering
Nx = 6     # number of state
Ny = 3     # number of observations
Nu = 2     # number of controls
x_hover = zeros(6)
u_hover = [0.5*m*g; 0.5*m*g]
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,u_hover),x_hover);
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x_hover,u),u_hover);
C = [I zeros(3,3)];

In [ ]:
#Set up visualization
using MeshCat
using RobotZoo: Quadrotor, PlanarQuadrotor
using CoordinateTransformations, Rotations, Colors, StaticArrays, RobotDynamics

function set_mesh!(vis, model::L;
        scaling=1.0, color=colorant"black"
    ) where {L <: Union{Quadrotor, PlanarQuadrotor}} 
    # urdf_folder = joinpath(@__DIR__, "..", "data", "meshes")
    urdf_folder = @__DIR__
    # if scaling != 1.0
    #     quad_scaling = 0.085 * scaling
    obj = joinpath(urdf_folder, "quadrotor_scaled.obj")
    if scaling != 1.0
        error("Scaling not implemented after switching to MeshCat 0.12")
    end
    robot_obj = MeshFileGeometry(obj)
    mat = MeshPhongMaterial(color=color)
    setobject!(vis["robot"]["geom"], robot_obj, mat)
    if hasfield(L, :ned)
        model.ned && settransform!(vis["robot"]["geom"], LinearMap(RotX(pi)))
    end
end

function visualize!(vis, model::PlanarQuadrotor, x::StaticVector)
    py,pz = x[1], x[2]
    θ = x[3]
    settransform!(vis["robot"], compose(Translation(0,py,pz), LinearMap(RotX(-θ))))
end

function visualize!(vis, model, tf::Real, X)
    fps = Int(round((length(X)-1)/tf))
    anim = MeshCat.Animation(vis; fps)
    for (k,x) in enumerate(X)
        atframe(anim, k) do
            x = X[k]
            visualize!(vis, model, SVector{6}(x)) 
        end
    end
    setanimation!(vis, anim)
end

In [ ]:
#LQR Hover Controller
Q = Array(1.0*I(Nx));
R = Array(0.1*I(Nu));
K = dlqr(A,B,Q,R)

xref = [0.0; 1.0; 0; 0; 0; 0]

function lqr_controller(x)
    return u_hover - K*(x-xref)
end

In [ ]:
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

function closed_loop(x0,N)
    xhist = zeros(Nx,N)
    uhist = zeros(Nu,N-1)
    xhist[:,1] .= x0
    for k = 1:(N-1)
        uhist[:,k] .= lqr_controller(xhist[:,k])
        xhist[:,k+1] .= xref + A*(xhist[:,k]-xref) + B*(uhist[:,k]-u_hover)
    end
    return xhist, uhist
end

In [ ]:
#Generate a bunch of trajectories from random initial conditions
N = 20
H = zeros((Ny+Nu)*(Nt-1),N)
for ℓ = 1:N
    x0 = xref + 0.2*randn(Nx)
    xtraj, utraj = closed_loop(x0, Nt);
    ytraj = C*xtraj[:,1:end-1];
    H[:,ℓ] = ([ytraj; utraj])[:]
end

In [ ]:
#Cloned controller
m = 2; #observation history
function cloned_controller(yhist)
    Y = [I zeros(m*Ny + (m-1)*Nu,(Ny*(Nt-m-1) + Nu*(Nt-m)))]
    U = [zeros(Nu,Ny*m + Nu*(m-1)) I zeros(Nu,(Ny+Nu)*(Nt-m-1))]
    w = (H'*Y'*Y*H)\(H'*Y'*yhist[:])
    u = U*H*w
    return u
end

In [ ]:
function closed_loop2(x0,N)
    xhist = zeros(Nx,N)
    uhist = zeros(Nu,N-1)
    xhist[:,1] .= x0
    for k = 1:(m-1)
        uhist[:,k] .= lqr_controller(xhist[:,k]);
        xhist[:,k+1] .= xref + A*(xhist[:,k]-xref) + B*(uhist[:,k]-u_hover)
    end
    for k = m:(N-1)
        yhist = [C*xhist[:,k-1]; uhist[:,k-1]; C*xhist[:,k]]
        uhist[:,k] .= cloned_controller(yhist)
        xhist[:,k+1] .= xref + A*(xhist[:,k]-xref) + B*(uhist[:,k]-u_hover)
    end
    return xhist, uhist
end

In [ ]:
vis = Visualizer()
model = PlanarQuadrotor()
set_mesh!(vis, model)
render(vis)

In [ ]:
#Rollout from random initial conditions
x0 = xref + 0.5*randn(Nx)
xhist1, uhist1 = closed_loop(x0, Nt);
xhist2, uhist2 = closed_loop2(x0, Nt);
xhist1-xhist2

In [ ]:
X2 = [SVector{6}(x) for x in eachcol(xhist2)];
visualize!(vis, model, thist[end], X2)